# Analyse Results for Transfer Learning tests
- Australian data
- Ensemble size is 20
- 20 Ensembles
- Compare full transfer and frozen layers with no transfer learning

## Imports

In [1]:
import glob
import json
import os
import pandas as pd
import random

from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import initialise
import common
from analysis_utils import calc_statistics, sort_file_list
from display_utils import display_frames

## Directories and other settings
- Update the model directories as required

In [2]:
models_dir = common.MODELS_DIR

model_dirs = [
    os.path.join(models_dir, 'europe_2000_adabn_cn'),
    os.path.join(models_dir, 'europe_2000_no-transfer'),
    os.path.join(models_dir, 'europe_2000_transfer'),
    os.path.join(models_dir, 'europe_2000_freeze-conv'),
    os.path.join(models_dir, 'europe_2000_freeze-all-conv'),
    os.path.join(models_dir, 'europe_2000_sourcerer'),
    os.path.join(models_dir, 'europe_2000_freeze-bn'),
    os.path.join(models_dir, 'europe_2000_sourcerer-reg'),
    os.path.join(models_dir, 'europe_2000_adabn')
]

output_dir = os.path.join(common.DATA_DIR, 'Results', 'Europe2000')
precision = 4       # floating point precision for saved predictions
test_names = ['Source-only', 'Target-only', 'Naive', 'Freeze-1st', 'Freeze-conv',
              'Sourcerer', 'Freeze-BN', 'Sourcerer-reg', 'AdaBN']

In [3]:
def get_stats(model_dir, epoch=''):
    test_stats = []
    test_dirs = sort_file_list(glob.glob(os.path.join(model_dir, f'test*')), 'test')
    for num, test_dir in enumerate(test_dirs):
        stats_ = pd.read_csv(os.path.join(test_dir, epoch, 'ensemble_stats.csv'), index_col=0)
        test_stats.append(stats_)
    return pd.concat(test_stats, ignore_index=True)

In [4]:
def get_preds(model_dir, epoch=''):
    test_preds = []
    test_dirs = sort_file_list(glob.glob(os.path.join(model_dir, f'test*')), 'test')
    for num, test_dir in enumerate(test_dirs):
        preds_ = pd.read_csv(os.path.join(test_dir, epoch, 'ensemble_predicts.csv'), index_col=0)
        test_preds.append(preds_)
    return pd.concat([test_preds[0].y] + [t.base for t in test_preds],
                     axis=1,
                     keys=['y']+[f'run_{i}' for i in range(len(test_preds))]).round(precision)

In [5]:
for idx, model_dir in enumerate(model_dirs):
    epoch = 'epoch0' if idx == 0 else ''
    file_name = test_names[idx]
    stats = get_stats(model_dir, epoch)
    stats_file = os.path.join(output_dir, f'{test_names[idx]}_stats.csv')
    stats.to_csv(stats_file)
    print(f"Stats file name: {stats_file}")
    preds = get_preds(model_dir, epoch)
    preds_file = os.path.join(output_dir, f'{test_names[idx]}_predicts.csv')
    preds.to_csv(preds_file)
    print(f"Preds file name: {preds_file}")


Stats file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Source-only_stats.csv
Preds file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Source-only_predicts.csv
Stats file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Target-only_stats.csv
Preds file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Target-only_predicts.csv
Stats file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Naive_stats.csv
Preds file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Naive_predicts.csv
Stats file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Freeze-1st_stats.csv
Preds file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe2000\Freeze-1st_predicts.csv
Stats file name: G:\My Drive\Postdoc\LFMC Project\Data\Domain_adaptation\Results\Europe200